# LEAST_SQUARES

## Overview
The `LEAST_SQUARES` function solves nonlinear least-squares problems using the SciPy optimization library. It fits a user-defined model function to observed data by minimizing the sum of squared residuals. This is useful for curve fitting, parameter estimation, and regression analysis directly in Excel.

## Usage
To use the `LEAST_SQUARES` function in Excel, enter it as a formula in a cell, specifying the model, data, and initial parameters:

```excel
=LEAST_SQUARES(model, xdata, ydata, p_zero, [bounds_lower], [bounds_upper], [method])
```

- `model` is a string representing the model function, e.g., "a * x + b" or "a * exp(b * x)". Use variables `x` and parameter names (e.g., `a`, `b`).
- `xdata` is a 2D list or column of x values.
- `ydata` is a 2D list or column of y values.
- `p_zero` is a 2D list or row of initial parameter guesses (e.g., [[1, 1]] for two parameters).
- `bounds_lower` and `bounds_upper` are optional 2D lists or rows specifying lower and upper bounds for each parameter.
- `method` is an optional string specifying the optimization method (e.g., "trf", "dogbox", "lm").

## Parameters
| Parameter      | Type     | Required | Description                                                                 |
|---------------|----------|----------|-----------------------------------------------------------------------------|
| model         | string   | Yes      | Model function as a string, e.g., "a * x + b". Use `x` and parameter names. |
| xdata         | 2D list  | Yes      | Input x values (independent variable).                                      |
| ydata         | 2D list  | Yes      | Observed y values (dependent variable).                                     |
| p_zero        | 2D list  | Yes      | Initial guesses for parameters.                                             |
| bounds_lower  | 2D list  | No       | Lower bounds for parameters.                                                |
| bounds_upper  | 2D list  | No       | Upper bounds for parameters.                                                |
| method        | string   | No       | Optimization method ("trf", "dogbox", "lm").                              |

## Return Value
| Return Value | Type   | Description                                  |
|--------------|--------|----------------------------------------------|
| Parameters   | 2D list| Fitted parameter values as a single row.      |

## Examples

### 1. Fit a straight line to data (y = a * x + b)

**Sample Input Data:**

| x | y |
|---|---|
| 1 | 2 |
| 2 | 4 |
| 3 | 6 |

```excel
=LEAST_SQUARES("a * x + b", A2:A4, B2:B4, {1, 1})
```
**Sample Output:**

| a   | b   |
|-----|-----|
| 2.0 | 0.0 |

### 2. Fit an exponential model (y = a * exp(b * x))

**Sample Input Data:**

| x | y  |
|---|----|
| 1 | 2.7|
| 2 | 7.4|
| 3 | 20.1|

```excel
=LEAST_SQUARES("a * exp(b * x)", A2:A4, B2:B4, {1, 1})
```
**Sample Output:**

| a     | b     |
|-------|-------|
| 1.0   | 1.0   |

### 3. Fit with parameter bounds (a >= 0, b >= 0)

**Sample Input Data:**

| x | y |
|---|---|
| 1 | 2 |
| 2 | 4 |
| 3 | 6 |

```excel
=LEAST_SQUARES("a * x + b", A2:A4, B2:B4, {1, 1}, {0, 0}, {10, 10})
```
**Sample Output:**

| a   | b   |
|-----|-----|
| 2.0 | 0.0 |

## Benefits
- Enables advanced curve fitting and regression in Excel without VBA or add-ins.
- Supports nonlinear models and parameter bounds.
- Useful for scientific, engineering, and business analysis.

In [1]:
import numpy as np
from scipy.optimize import least_squares as scipy_least_squares
import math
SAFE_GLOBALS = {k: getattr(math, k) for k in dir(math) if not k.startswith("_")}
SAFE_GLOBALS["np"] = np
SAFE_GLOBALS["numpy"] = np
SAFE_GLOBALS["exp"] = np.exp
SAFE_GLOBALS["log"] = np.log
SAFE_GLOBALS["sin"] = np.sin
SAFE_GLOBALS["cos"] = np.cos
SAFE_GLOBALS["tan"] = np.tan
SAFE_GLOBALS["abs"] = abs
SAFE_GLOBALS["pow"] = pow

def least_squares(model, xdata, ydata, p_zero, bounds_lower=None, bounds_upper=None, method=None):
    try:
        x = np.array(xdata).flatten()
        y = np.array(ydata).flatten()
        p_zero = np.array(p_zero).flatten()
        n_params = len(p_zero)
        import re
        param_names = re.findall(r'\b[a-zA-Z_]\w*\b', model)
        param_names = [name for name in param_names if name not in ("x", "exp", "log", "sin", "cos", "tan", "abs", "pow")]
        param_names = list(dict.fromkeys(param_names))
        if len(param_names) != n_params:
            return f"Number of initial guesses (p_zero) does not match number of parameters in model: {param_names}"
        if bounds_lower is not None and bounds_upper is not None:
            bounds = (np.array(bounds_lower).flatten(), np.array(bounds_upper).flatten())
        else:
            bounds = (-np.inf, np.inf)
        def residuals(params):
            local_dict = dict(zip(param_names, params))
            local_dict["x"] = x
            try:
                y_pred = eval(model, SAFE_GLOBALS, local_dict)
            except Exception as e:
                raise ValueError(f"Model evaluation error: {e}")
            return y_pred - y
        lsq_method = method if method is not None else 'trf'
        if lsq_method not in ('trf', 'dogbox', 'lm'):
            return "`method` must be 'trf', 'dogbox' or 'lm'."
        result = scipy_least_squares(residuals, p_zero, bounds=bounds, method=lsq_method)
        if not result.success:
            return f"Fit failed: {result.message}"
        return [result.x.tolist()]
    except Exception as e:
        return str(e)

In [2]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()
def test_demo_linear_fit():
    model = "a * x + b"
    xdata = [[1], [2], [3]]
    ydata = [[2], [4], [6]]
    p_zero = [[1, 1]]
    result = least_squares(model, xdata, ydata, p_zero)
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert all(isinstance(x, (float, int)) for x in result[0])
    assert abs(result[0][0] - 2.0) < 1e-2
    assert abs(result[0][1] - 0.0) < 1e-2

def test_demo_exponential_fit():
    model = "a * exp(b * x)"
    xdata = [[1], [2], [3]]
    ydata = [[2.7], [7.4], [20.1]]
    p_zero = [[1, 1]]
    result = least_squares(model, xdata, ydata, p_zero)
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert all(isinstance(x, (float, int)) for x in result[0])
    assert abs(result[0][0] - 1.0) < 1e-1
    assert abs(result[0][1] - 1.0) < 1e-1

def test_demo_bounds():
    model = "a * x + b"
    xdata = [[1], [2], [3]]
    ydata = [[2], [4], [6]]
    p_zero = [[1, 1]]
    bounds_lower = [[0, 0]]
    bounds_upper = [[10, 10]]
    result = least_squares(model, xdata, ydata, p_zero, bounds_lower, bounds_upper)
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert all(isinstance(x, (float, int)) for x in result[0])
    assert abs(result[0][0] - 2.0) < 1e-2
    assert abs(result[0][1] - 0.0) < 1e-2

ipytest.run()

...                                                                                          [100%]
3 passed in 0.02s


<ExitCode.OK: 0>

In [9]:
# Interactive Demo
import gradio as gr

# Gradio Interface
examples = [
    [
        "a * x + b",
        [[1], [2], [3]],
        [[2], [4], [6]],
        [[1, 1]],
        [[0, 0]],
        [[10, 10]],
        "trf"
    ],
    [
        "a * exp(b * x)",
        [[1], [2], [3]],
        [[2.7], [7.4], [20.1]],
        [[1, 1]],
        [[0, 0]],
        [[10, 10]],
        "trf"
    ],
    [
        "a * x + b",
        [[1], [2], [3]],
        [[2], [4], [6]],
        [[1, 1]],
        [[0, 0]],
        [[10, 10]],
        "trf"
    ]
]

demo = gr.Interface(
    fn=least_squares,
    inputs=[
        gr.Textbox(label="Model", value="a * x + b"),
        gr.Dataframe(label="xdata", row_count=3, col_count=1, value=[[1], [2], [3]], type="array"),
        gr.Dataframe(label="ydata", row_count=3, col_count=1, value=[[2], [4], [6]], type="array"),
        gr.Dataframe(label="Initial Guesses (p_zero)", row_count=1, col_count=2, value=[[1, 1]], type="array"),
        gr.Dataframe(label="Lower Bounds (optional)", row_count=1, col_count=2, value=[[0, 0]], type="array"),
        gr.Dataframe(label="Upper Bounds (optional)", row_count=1, col_count=2, value=[[10, 10]], type="array"),
        gr.Textbox(label="Method (optional)", value="trf"),
    ],
    outputs=gr.Dataframe(label="Fitted Parameters"),
    examples=examples,
    description="Fit a nonlinear model to data using least squares. Set the model, data, and initial guesses. Leave method blank for default.",
    flagging_mode="never",
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.
